In [ ]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

filterKey = "".join(sorted("english"))

stopWords = set(word.strip().lower() for word in open("stop_words_en.txt"))

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:        
        word = word.lower()
        if word not in stopWords: 
            key = "".join(sorted(word))        
            print "%s\t%s" % (key, word)

In [ ]:
%%writefile reducer.py

import sys
from sets import Set

current_key = None
word_sum = 0
words = Set()

In [ ]:
%%writefile -a reducer.py


for line in sys.stdin:
    try:
        key, word = line.strip().split('\t', 1)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            if len(words)>1:
                print "%s\t%d\t%d\t%s" % (current_key, word_sum, len(words), ",".join(sorted(words)))
        word_sum = 0
        current_key = key
        words = Set()
    word_sum += 1
    words.add(word)

if current_key:
    if len(words)>1:
        print "%s\t%d\t%d\t%s" % (current_key, word_sum, len(words), ",".join(sorted(words)))

In [ ]:
%%writefile mapper1.py

import sys

filterKey = "".join(sorted("english"))


for line in sys.stdin:
    try:
        key, text = line.strip().split('\t', 1)
    except ValueError as e:
        continue
    if key == filterKey:         
        print "%s" % text

In [ ]:
%%bash

GROUPS_DIR="word_groups"
RESULT_DIR="group_res"
NUM_REDUCERS=8

hdfs dfs -rm -r -skipTrash ${GROUPS_DIR} > /dev/null

hdfs dfs -rm -r -skipTrash ${RESULT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming word group" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py,/datasets/stop_words_en.txt \
    -mapper "python mapper.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${GROUPS_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.name="Streaming group filter" \
    -D mapreduce.job.reduces=1 \
    -files mapper1.py, \
    -mapper "python mapper1.py" \
    -input ${GROUPS_DIR} \
    -output ${RESULT_DIR} > /dev/null

hdfs dfs -cat ${RESULT_DIR}/part-00000 | head